# 랩 3: AgentCore Gateway로 에이전트에 도구를 안전하게 연결

## 개요

이 랩에서는 Amazon Bedrock Gateway를 사용하여 조직에서 사용 가능한 도구를 고객 지원 에이전트와 통합하는 방법을 배웁니다.

[Model Context Protocol (MCP)](https://modelcontextprotocol.io/docs/getting-started/intro)는 애플리케이션이 대형 언어 모델(LLM)에 도구와 컨텍스트를 제공하는 방법을 표준화하는 개방형 프로토콜입니다.

[Amazon Bedrock 에이전트 Core 게이트웨이](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/게이트웨이.html)를 사용하면 개발자가 몇 줄의 코드만으로 API, Lambda 함수, 기존 서비스를 MCP 호환 도구로 변환하고 게이트웨이 엔드포인트를 통해 에이전트에서 사용할 수 있도록 만들 수 있습니다.


**워크샵 여정:**

- **랩 1 (완료):** 에이전트 프로토타입 생성 - 기능적인 고객 지원 에이전트 구축
- **랩 2 (완료):** 메모리로 향상 - 대화 컨텍스트와 개인화 추가
- **랩 3 (현재):** 게이트웨이 및 아이덴티티로 확장 - 에이전트 간 도구 안전하게 공유
- **랩 4:** 프로덕션 배포 - 관찰 가능성을 갖춘 AgentCore 런타임 사용
- **랩 5:** 사용자 인터페이스 구축 - 고객 대면 애플리케이션 생성


### AgentCore Gateway와 도구 공유가 중요한 이유

현재 상태 (랩 1-2): 각 에이전트가 자체 도구 사본을 가지고 있습니다. 이는 확장 가능하지 않은 방식으로 다음과 같은 문제를 야기합니다:

- 서로 다른 에이전트 간 코드 중복
- 일관성 없는 도구 동작과 유지보수 오버헤드
- 중앙화된 보안이나 액세스 제어 없음
- 여러 사용 사례로 확장하기 어려움

이 랩 후에는 다음을 서비스할 수 있는 중앙화되고 재사용 가능한 도구를 갖게 됩니다:

- 고객 지원 에이전트 (현재 사용 사례)
- 영업 에이전트 (동일한 제품 정보와 고객 데이터 필요)
- 재고 에이전트 (동일한 제품 정보와 보증 확인 필요)
- 반품 처리 에이전트 (반품 정책과 고객 프로필 필요)

그리고 기타 사용 사례들.

### Adding secure 인증 with AgentCore 아이덴티티

Additionally, AgentCore 게이트웨이 requires you to securely authenticate both inbound and outbound connections. [AgentCore 아이덴티티](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/아이덴티티.html) provides seamless 에이전트 아이덴티티 and access management across AWS services and third-party applications such as Slack and Zoom while supporting any standard 아이덴티티 providers such as Okta, Entra, and Amazon Cognito. In this lab we will see how AgentCore 게이트웨이 integrates with AgentCore 아이덴티티 to provide secure connections via inbound and outbound 인증. 

For the inbound 인증, the AgentCore 게이트웨이 analyzes the OAuth 토큰 passed during invocation to decide allow or deny the access to a 도구 in the 게이트웨이. If a 도구 needs access to external 리소스, the AgentCore 게이트웨이 can use outbound 인증 via API Key, IAM or OAuth 토큰 to allow or deny the access to the external resource.

During the inbound 인가 flow, an 에이전트 or the MCP 클라이언트 calls an MCP 도구 in the AgentCore 게이트웨이 adding an OAuth access 토큰 (generated from the user’s IdP). AgentCore 게이트웨이 then validates the OAuth access 토큰 and performs inbound 인가.

If the 도구 running in AgentCore 게이트웨이 needs to access external 리소스, OAuth will retrieve credentials of downstream 리소스 using the resource credential provider for the 게이트웨이 대상. AgentCore 게이트웨이 pass the 인가 credentials to the caller to get access to the downstream API.


## 아키텍처 for Lab 3

<div style="text-align:left">
    <img src="images/architecture_lab3_gateway.png" width="75%"/>
</div>

*웹 검색 도구는 이제 보안 ID 기반 액세스 제어를 통해 AgentCore Gateway에 중앙화되었습니다. 여러 에이전트와 사용 사례가 동일한 도구를 안전하게 공유할 수 있습니다. 또한 다른 애플리케이션용으로 구축된 `check_warranty()` 및 `get_customer_profile()` 도구를 재사용할 것입니다. `get_return_policy()` 및 `get_product_info()`는 고객 지원 사용 사례에 특화되어 있으므로 로컬 도구로 유지됩니다.* 

### 주요 기능
- **AWS Lambda 함수 원활한 통합:** 이 예제는 Amazon Bedrock AgentCore Gateway를 사용하여 항목의 보증을 확인하고 고객 프로필을 가져오는 기존 AWS Lambda 함수와 에이전트를 통합하는 방법을 보여줍니다.
- **인바운드 인증으로 게이트웨이 엔드포인트 보안**: 유효한 JWT 토큰을 제공하는 에이전트만 엔드포인트에 연결하여 도구를 사용할 수 있습니다
- **MCP 엔드포인트를 사용하도록 에이전트 구성**: 에이전트는 유효한 JWT 토큰을 받아 AgentCore Gateway에서 제공하는 MCP 엔드포인트에 연결하는 데 사용합니다

## 전제조건

* Python 3.12+
* AWS credentials configured
* Anthropic Claude 3.7 enabled on [Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)
* 랩 2 완료: 고객 지원 에이전트에 메모리 추가
* 이러한 리소스는 AWS 워크샵 계정 내에서 생성됩니다
    - AWS Lambda 함수 
    - AWS Lambda Execution IAM Role
    - AgentCore 게이트웨이 IAM Role
    - AWS Lambda 함수에서 사용하는 DynamoDB 테이블
    - Cognito 사용자 풀 및 사용자 풀 클라이언트
#### AWS 워크샵 계정을 사용하지 않는 경우?

**참고:** 이를 자율 진행 랩으로 실행하는 경우 워크샵 자율 진행 단계에 표시된 대로 CloudFormation 리소스를 생성해야 합니다. 그렇지 않은 경우 아래 코드 세그먼트의 주석을 해제하고 실행하세요

In [1]:
#!bash scripts/prereq.sh

## 1단계: 필요한 라이브러리 설치 및 가져오기

In [ ]:
# Install required packages
%pip install strands-agents "boto3>=1.39.15" strands-agents-tools bedrock_agentcore ddgs -q

In [ ]:
# Import libraries
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
import os
import sys
import boto3
import json
from bedrock_agentcore.identity.auth import requires_access_token
from mcp.client.streamable_http import streamablehttp_client
import requests

from scripts.utils import get_ssm_parameter, put_ssm_parameter, load_api_spec, get_cognito_client_secret

sts_client = boto3.client('sts')

# Get AWS account details
REGION = boto3.session.Session().region_name

gateway_client = boto3.client(
    "bedrock-agentcore-control",
    region_name=REGION,
)

print("✅ Libraries imported successfully!")

## 2단계: 기존 고객 데이터에 액세스할 수 있는 도구를 에이전트에 제공
AgentCore Gateway는 세 가지 주요 방법으로 에이전트 도구 통합을 단순화합니다:

범용 MCP 지원: AgentCore Gateway의 MCP 표준을 통해 노출하여 도구를 모든 에이전트 프레임워크와 즉시 호환되도록 만듭니다.

간단한 REST 통합: 기존 REST 서비스를 AgentCore 게이트웨이 대상으로 추가하기만 하면 에이전트 도구로 변환합니다.

Lambda 유연성: 모든 API를 호출할 수 있는 MCP 엔드포인트로 Lambda 함수를 노출합니다. 여기서는 보증 상태를 확인하는 함수로 시연됩니다.

AgentCore Gateway는 호출할 도구의 이름으로 Lambda 컨텍스트를 채우고, 도구에 전달된 매개변수는 Lambda 이벤트에서 제공됩니다:

```
extended_tool_name = context.client_context.custom["bedrockAgentCoreToolName"]
resource = extended_tool_name.split("___")[1]
```

[Lambda 함수](./prerequisite/Lambda/python/lambda_function.py)

```
def lambda_handler(event, context):
    if get_tool_name(event) == "check_warranty_status":
        serial_number = get_named_parameter(event=event, name="serial_number")
        customer_email = get_named_parameter(event=event, name="customer_email")

        warranty_status = check_warranty_status(serial_number, customer_email)
        return {"statusCode": 200, "body": warranty_status}
```

## 3단계: 웹 검색 도구를 MCP로 변환
이제 AgentCore Gateway를 사용하여 MCP 서버를 개발하고 있으므로, 여러 에이전트에서 사용할 것으로 생각되는 모든 도구를 MCP화할 수 있습니다. 이러한 도구 중 하나는 랩 1에서 구축한 것과 같은 웹 검색 도구일 수 있습니다. 결과적으로 랩 1의 웹 검색 도구를 AgentCore 게이트웨이 내의 Lambda 도구로 변환했습니다:

[웹 검색 Lambda](./prerequisite/Lambda/python/web_search.py)
```
from ddgs 가져오기 DDGS


def web_search(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """업데이트된 정보를 웹에서 검색합니다.
    
    Args:
        keywords (str): 검색 쿼리 키워드.
        region (str): 검색 지역: wt-wt, us-en, uk-en, ru-ru 등.
        max_results (int): 반환할 최대 결과 수.
        
    Returns:
        검색 결과가 포함된 딕셔너리 목록.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "검색 결과를 찾을 수 없습니다."
    except Exception as e:
        return f"검색 오류: {str(e)}"


print("✅ 웹 검색 도구 준비 완료")
```

## 4단계: 함수 정의 메타데이터 생성
마지막으로 Lambda 함수에서 구현한 도구를 설명하는 도구 스키마를 작성해야 합니다.

이 파일은 이미 [prerequisite/Lambda/api_spec.json](./prerequisite/Lambda/api_spec.json)에 정의되어 있습니다.

```
[
    {
        "name": "check_warranty_status",
        "description": "제품의 일련번호를 사용하여 보증 상태를 확인하고 선택적으로 이메일을 통해 검증",
        "inputSchema": {
            "type": "object",
            "properties": {
                "serial_number": {
                    "type": "string"
                },
                "customer_email": {
                    "type": "string"
                }
            },
            "required": [
                "serial_number"
            ]
        }
    },
    {
        "name": "web_search",
        "description": "DuckDuckGo를 사용하여 업데이트된 정보를 웹에서 검색",
        "inputSchema": {
            "type": "object",
            "properties": {
                "keywords": {
                    "type": "string",
                    "description": "검색 쿼리 키워드"
                },
                "region": {
                    "type": "string",
                    "description": "검색 지역 (예: us-en, uk-en, ru-ru)"
                },
                "max_results": {
                    "type": "integer",
                    "description": "반환할 최대 결과 수"
                }
            },
            "required": [
                "keywords"
            ]
        }
    }
]
```

## 5단계. AgentCore 게이트웨이 생성

이제 Lambda 함수를 MCP 호환 엔드포인트로 노출하는 AgentCore Gateway를 생성해보겠습니다.

도구를 호출할 권한이 있는 호출자를 검증하려면 인바운드 인증을 구성해야 합니다.

인바운드 인증은 MCP 서버의 표준인 OAuth 인증을 사용하여 작동합니다. OAuth를 사용하면 클라이언트 애플리케이션이 게이트웨이를 사용하기 전에 OAuth 인증자로 인증해야 합니다. 클라이언트는 런타임에 사용되는 액세스 토큰을 받게 됩니다.

OAuth 검색 서버와 클라이언트 ID를 지정해야 합니다. 워크샵과 함께 제공된 CloudFormation은 이미 Cognito UserPool과 UserPoolClient를 프로비저닝했으며 검색 URL과 클라이언트 ID를 전용 SSM 매개변수에 저장했습니다.

In [ ]:
gateway_name = "customersupport-gw"

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            get_ssm_parameter("/app/customersupport/agentcore/machine_client_id")
        ],
        "discoveryUrl": get_ssm_parameter("/app/customersupport/agentcore/cognito_discovery_url")
    }
}

try:
    # create new gateway
    print(f"Creating gateway in region {REGION} with name: {gateway_name}")

    create_response = gateway_client.create_gateway(
        name=gateway_name,
        roleArn= get_ssm_parameter("/app/customersupport/agentcore/gateway_iam_role"),
        protocolType="MCP",
        authorizerType="CUSTOM_JWT",
        authorizerConfiguration=auth_config,
        description="Customer Support AgentCore Gateway",
    )

    gateway_id = create_response["gatewayId"]

    gateway = {
        "id": gateway_id,
        "name": gateway_name,
        "gateway_url": create_response["gatewayUrl"],
        "gateway_arn": create_response["gatewayArn"],
    }
    put_ssm_parameter("/app/customersupport/agentcore/gateway_id", gateway_id)

    print(f"✅ Gateway created successfully with ID: {gateway_id}")

except Exception as e:
    # If gateway exists, collect existing gateway ID from SSM
    existing_gateway_id = get_ssm_parameter("/app/customersupport/agentcore/gateway_id")
    print(f"Found existing gateway with ID: {existing_gateway_id}")
    
    # Get existing gateway details
    gateway_response = gateway_client.get_gateway(gatewayIdentifier=existing_gateway_id)
    gateway = {
        "id": existing_gateway_id,
        "name": gateway_response["name"],
        "gateway_url": gateway_response["gatewayUrl"],
        "gateway_arn": gateway_response["gatewayArn"],
    }
    gateway_id = gateway['id']

## 6단계. Lambda 함수 대상 추가
이제 [prerequisite/Lambda/api_spec.json](./prerequisite/Lambda/api_spec.json)에서 이전에 정의한 함수 정의를 사용하여 에이전트 게이트웨이 내에 Lambda 대상을 생성합니다. 이는 게이트웨이가 호스팅할 도구를 정의합니다.

게이트웨이를 사용하면 게이트웨이에 여러 대상을 연결할 수 있으며 언제든지 게이트웨이에 연결된 대상/도구를 변경할 수 있습니다. 각 대상은 고유한 자격 증명 제공자를 가질 수 있지만, 게이트웨이는 단일 MCP URL이 되어 수많은 API에 걸쳐 에이전트에 대한 모든 관련 도구에 액세스할 수 있게 합니다.

In [ ]:
def load_api_spec(file_path: str) -> list:
    with open(file_path, "r") as f:
        data = json.load(f)
        
    if not isinstance(data, list):
        raise ValueError("Expected a list in the JSON file")
    return data

try:
    api_spec_file = "./prerequisite/lambda/api_spec.json"

    # Validate API spec file exists
    if not os.path.exists(api_spec_file):
        print(f"❌ API specification file not found: {api_spec_file}")
        sys.exit(1)

    api_spec = load_api_spec(api_spec_file)
 
    # Use Cognito for Inbound OAuth to our Gateway
    lambda_target_config = {
        "mcp": {
            "lambda": {
                "lambdaArn": get_ssm_parameter("/app/customersupport/agentcore/lambda_arn"),
                "toolSchema": {"inlinePayload": api_spec},
            }
        }
    }


    # Create gateway target
    credential_config = [{"credentialProviderType": "GATEWAY_IAM_ROLE"}]

    create_target_response = gateway_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name="LambdaUsingSDK",
        description="Lambda Target using SDK",
        targetConfiguration=lambda_target_config,
        credentialProviderConfigurations=credential_config,
    )

    print(f"✅ Gateway target created: {create_target_response['targetId']}")

except Exception as e:
    print(f"❌ Error creating gateway target: {str(e)}")

## 7단계: 지원 에이전트에 새로운 MCP 기반 도구 추가
여기서는 Cognito의 인증 토큰을 Strands SDK의 MCPClient에 통합하여 Strands Agent와 통합할 MCP 서버 객체를 생성합니다

In [ ]:
def get_token(client_id: str, client_secret: str, scope_string: str, url: str) -> dict:
    try:
        headers = {"Content-Type": "application/x-www-form-urlencoded"}
        data = {
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret,
            "scope": scope_string,

        }
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status()
        return response.json()

    except requests.exceptions.RequestException as err:
        return {"error": str(err)}

## 7.1단계. 보안 MCP 클라이언트 객체 설정

In [ ]:
gateway_access_token = get_token(
    get_ssm_parameter("/app/customersupport/agentcore/machine_client_id"),
    get_cognito_client_secret(),
    get_ssm_parameter("/app/customersupport/agentcore/cognito_auth_scope"),
    get_ssm_parameter("/app/customersupport/agentcore/cognito_token_url"))

print(f"Gateway Endpoint - MCP URL: {gateway['gateway_url']}")

# Set up MCP client
mcp_client = MCPClient(
    lambda: streamablehttp_client(
        gateway['gateway_url'],
        headers={"Authorization": f"Bearer {gateway_access_token['access_token']}"},
    )
)

## 7.2단계. MCP 클라이언트를 사용하여 에이전트의 도구에 액세스
이제 구축한 AgentCore Gateway와 이전 랩의 리소스를 사용하여 Strands Agent를 생성합니다. 이제 에이전트는 Strands Agent를 통한 로컬 도구와 AgentCore Gateway를 통한 MCP 도구를 혼합하여 사용합니다

In [ ]:
from lab_helpers.lab1_strands_agent import get_product_info, get_return_policy, SYSTEM_PROMPT
from lab_helpers.lab2_memory import CustomerSupportMemoryHooks,create_or_get_memory_resource 
import uuid
from bedrock_agentcore.memory import MemoryClient

memory_client = MemoryClient(region_name=REGION)

memory_id = create_or_get_memory_resource()
SESSION_ID = str(uuid.uuid4())
CUSTOMER_ID = "customer_001"
memory_hooks = CustomerSupportMemoryHooks(memory_id, memory_client, CUSTOMER_ID, SESSION_ID)

# Initialize the Bedrock model
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(
    model_id=model_id,
    temperature=0.3,  # Balanced between creativity and consistency
    region_name=REGION
)

try:
    mcp_client.start()
except Exception as e:
    print(f"Error initializing agent: {str(e)}")

tools = (
            [
                get_product_info,
                get_return_policy,
            ]
            + mcp_client.list_tools_sync()
        )

# Create the customer support agent
agent = Agent(
    model=model,
    tools=tools,
    hooks=[memory_hooks],
    system_prompt=SYSTEM_PROMPT
)

print("✅ Customer support agent created successfully!")

## 8단계: 기존 API에 대한 MCP 도구 액세스로 에이전트 테스트”

모든 기능이 올바르게 작동하는지 확인하기 위해 샘플 쿼리로 에이전트를 테스트해보겠습니다.

In [ ]:
test_prompts = [
    # Warranty Checks
    "List all of your tools",
    "I have a Gaming Console Pro device , I want to check my warranty status, warranty serial number is MNO33333333.",
    "What are the warranty support guidelines?",
    "How can I fix Lenovo Thinkpad with a blue screen",
    "Tell me detailed information about the technical documentation on installing a new CPU"
]

# Function to test the agent
def test_agent_responses(agent, prompts):
    for i, prompt in enumerate(prompts, 1):
        print(f"\nTest Case {i}: {prompt}")
        print("-" * 50)
        try:
            response = agent(prompt)
        except Exception as e:
            print(f"Error: {str(e)}")
        print("-" * 50)

# Run the tests
test_agent_responses(agent, test_prompts)

print("\\n✅ Basic testing completed!")


### 축하합니다! 🎉


랩 3: AgentCore Gateway로 에이전트에 도구를 안전하게 연결하기를 성공적으로 완료했습니다!

달성한 내용:

##### 도구 중앙화 및 재사용성:

- 로컬 도구에서 중앙화된 AgentCore Gateway로 웹 검색 마이그레이션
- 기존 엔터프라이즈 Lambda 함수 통합 (보증 확인, 고객 프로필)
- 여러 에이전트 유형이 액세스할 수 있는 공유 도구 인프라 생성

##### 엔터프라이즈급 보안:

- Cognito 통합을 통한 JWT 기반 인증 구현
- 게이트웨이 액세스를 위한 보안 인바운드 인가 구성
- 도구 사용을 위한 ID 기반 액세스 제어 구축

##### 확장 가능한 아키텍처 기반:

- 여러 사용 사례(고객 지원, 영업, 반품 처리)를 서비스하는 재사용 가능한 도구 구축
- 서로 다른 에이전트 간 코드 중복 제거
- 도구 업데이트 및 유지보수를 위한 중앙화된 관리 생성

##### 현재 한계 (다음에 해결할 예정입니다!):

- **로컬 개발 환경** - 여전히 노트북에서 실행 중이며 프로덕션 준비가 되지 않음
- **제한된 관찰 가능성** - 에이전트 동작 및 성능에 대한 포괄적인 모니터링 없음
- **수동 확장** - 증가된 부하나 여러 동시 사용자를 자동으로 처리할 수 없음

##### 다음 단계: 랩 4 - AgentCore Runtime으로 프로덕션에 배포

랩 4에서는 프로토타입을 다음과 같은 프로덕션 준비 시스템으로 변환합니다:

- 확장 가능한 에이전트 배포를 위한 AgentCore Runtime
- 메트릭, 로깅, 추적을 통한 포괄적인 관찰 가능성
- 실제 트래픽을 처리하는 자동 확장 기능

### 리소스
- [Amazon Bedrock 에이전트 Core 게이트웨이](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/게이트웨이.html)
- [Strands Agents Documentation](https://github.com/strands-agents/sdk-python)
- [Official Customer Support Sample](https://github.com/awslabs/amazon-bedrock-agentcore-samples/tree/main/02-use-cases/customer-support-assistant)